In [32]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

In [54]:
import gym_tetris
import random
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT

In [55]:
# class JoypadSpaceMixin(JoypadSpace):
#     def step(self, action):
#         n_state, reward, done, info = super().step(action)
#         return n_state, reward, done, info

In [56]:
env = gym_tetris.make('TetrisA-v0')
env = JoypadSpace(env, MOVEMENT)
states = env.observation_space.shape[0]
actions = env.action_space.n
states, actions

(240, 12)

In [59]:
# episodes = 10
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         env.render()
#         n_state, reward, done, info = env.step(env.action_space.sample())
#         score+=reward
#         print("info:")
#         print(info)
#         print()
#     print('Episode:{} Score:{}'.format(episode, score))

In [60]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [61]:
def build_model(states, actions):
    model = tensorflow.keras.models.Sequential()
    model.add(Flatten(input_shape=(1, 240, 256, 3)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [62]:
model = build_model(states, actions)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 184320)            0         
                                                                 
 dense_15 (Dense)            (None, 24)                4423704   
                                                                 
 dense_16 (Dense)            (None, 24)                600       
                                                                 
 dense_17 (Dense)            (None, 12)                300       
                                                                 
Total params: 4,424,604
Trainable params: 4,424,604
Non-trainable params: 0
_________________________________________________________________


In [63]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [64]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 6132s 613ms/step - reward: 0.0000e+00
Interval 2 (10000 steps performed)
10000/10000 [==============================] - 5933s 593ms/step - reward: 0.0000e+00
1 episodes - episode_reward: 0.000 [0.000, 0.000] - loss: 26478.633 - mae: 3.258 - mean_q: 4.990 - number_of_lines: 0.000 - score: 0.000 - board_height: 10.376

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 5950s 595ms/step - reward: 1.0000e-04
2 episodes - episode_reward: 0.500 [0.000, 1.000] - loss: 0.000 - mae: 1.184 - mean_q: 1.305 - number_of_lines: 0.000 - score: 0.625 - board_height: 8.066

Interval 4 (30000 steps performed)
 3841/10000 [==========>...................] - ETA: 1:03:13 - reward: 0.0000e+00

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)